* This is my first major take on image classification competition and I have tried to put forth all the learnings from my deep learning course.

* References:

1.https://www.kaggle.com/abhishek/very-simple-pytorch-training-0-59

2.https://www.kaggle.com/super13579/pytorch-nn-cyclelr-k-fold-0-897-lightgbm-0-899

3.https://www.kaggle.com/artgor/basic-eda-and-baseline-pytorch-model/data

4.https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter

5.https://github.com/anandsaha/pytorch.cyclic.learning.rate/blob/master/cls.py

6.Various discussions in the forum

In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import time
import seaborn as sns
import random
import sys
%matplotlib inline


import cv2
import torch
from PIL import Image,ImageFile
from os.path import isfile, join, abspath, exists, isdir, expanduser
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset,DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
import albumentations
from albumentations import torch as AT
from torch.optim.optimizer import Optimizer


from sklearn.model_selection import train_test_split,StratifiedKFold,KFold



import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import gc
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## early stopping package:
package='../input/earlystoppingpytorch/early-stopping-pytorch/early-stopping-pytorch'
sys.path.append(package)
from pytorchtools import EarlyStopping

['pretrained-pytorch-models', 'earlystoppingpytorch', 'aptos2019-blindness-detection']


In [2]:

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
#Import the labels,
train_labels=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
test=pd.read_csv("../input/aptos2019-blindness-detection/test.csv")
sample=pd.read_csv("../input/aptos2019-blindness-detection/sample_submission.csv")

In [22]:
train_labels.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


In [23]:
len(train_labels['id_code'])

3662

In [24]:
train_labels['diagnosis'].value_counts()

0    1805
2     999
1     370
4     295
3     193
Name: diagnosis, dtype: int64

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

The number of images with No DR is 1805 followed by Moderate DR and Mild DR.

In [ ]:
test.head()

In [ ]:
len(test['id_code'])

In [ ]:
len(sample)

In [4]:
NUM_CLASS=5
BATCH_SIZE=32
IMAGE_SIZE=224
data_dir="../input/aptos2019-blindness-detection/"


In [ ]:
#train_x,valid_x,train_y,valid_y=train_test_split(train_labels,train_labels['diagnosis'],test_size=0.2,shuffle=train_labels['diagnosis'])

In [ ]:
#train_x.shape,train_y.shape,valid_x.shape,valid_y.shape

In [ ]:
#Inspired from - https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_name = '{}.png'.format(df.iloc[i,0])  
        image_id = df.iloc[i,1]
        img_path=data_dir+'/train_images/'
        img_path=join(img_path,image_name)
        
        img = Image.open(img_path)
        
        
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_labels)

In [5]:
## Import the data:
class eye(Dataset):
    def __init__(self,labels,directory,subset=False,transform=None):
        self.labels=labels
        self.directory=directory
        self.transform=transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        img_name='{}.png'.format(self.labels.iloc[idx,0])  
        full_image_path=join(self.directory,img_name)
        #print(f'\nImage path:{full_image_path}')
        #img = cv2.imread(img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #image = self.transform(img)
        #image = image['image']
        image=Image.open(full_image_path)
        image_resize = image.resize((224, 224), resample=Image.BILINEAR)
        image=self.transform(image)
        #image=image['image']
        
        image_label=self.labels.iloc[idx,1:].as_matrix().astype('float')
        image_label=np.argmax(image_label)
        
        
            
        return [image,image_label]
    

In [38]:
class RetinopathyDatasetTest(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/test_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        return {'image': image}

In [ ]:
# class eyeTest(Dataset):
#     def __init__(self,data,transform=None):
#         self.data = data
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         img_name='{}.png'.format(self.data.iloc[idx,0])  
#         full_image_path=join(data_dir+'/test_images/',img_name)
#         image=Image.open(full_image_path)
#         image = image.resize((299, 299), resample=Image.BILINEAR)
#         image=self.transform(image)
#         return {'image': image}

In [6]:
train_trans=transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(degrees=10),transforms.RandomVerticalFlip(),transforms.Resize((224,224)),transforms.ToTensor()])
                                #transforms.Normalize(mean=[0.485,0.485,0.485],std=[0.485,0.485,0.485])])
test_trans=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
                                #transforms.RandomHorizontalFlip(),
                                #transforms.RandomRotation(),
                               # transforms.Normalize(mean=[0.485,0.485,0.485],std=[0.485,0.485,0.485])])
# train_ds=eye(train_x,data_dir+'/train_images/',transform=train_trans)
# valid_ds=eye(valid_x,data_dir+'/train_images/',transform=train_trans)
# test_ds=eye(sample,data_dir+'/test_images/',transform=train_trans)

# train_dl=DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
# valid_dl=DataLoader(valid_ds,batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
# test_dl=DataLoader(test_ds,shuffle=True,num_workers=4)


# Model Building

In [7]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
#taken from https://github.com/anandsaha/pytorch.cyclic.learning.rate/blob/master/cls.py
# This code is from https://github.com/thomasjpfan/pytorch/blob/401ec389db2c9d2978917a6e4d1101b20340d7e7/torch/optim/lr_scheduler.py
# This code is under review at PyTorch and is to be merged eventually to make CLR available to all.
# Tested with pytorch 0.2.0




class CyclicLR(object):
    """Sets the learning rate of each parameter group according to
    cyclical learning rate policy (CLR). The policy cycles the learning
    rate between two boundaries with a constant frequency, as detailed in
    the paper `Cyclical Learning Rates for Training Neural Networks`_.
    The distance between the two boundaries can be scaled on a per-iteration
    or per-cycle basis.
    Cyclical learning rate policy changes the learning rate after every batch.
    `batch_step` should be called after a batch has been used for training.
    To resume training, save `last_batch_iteration` and use it to instantiate `CycleLR`.
    This class has three built-in policies, as put forth in the paper:
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each
        cycle iteration.
    This implementation was adapted from the github repo: `bckenstler/CLR`_
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        base_lr (float or list): Initial learning rate which is the
            lower boundary in the cycle for eachparam groups.
            Default: 0.001
        max_lr (float or list): Upper boundaries in the cycle for
            each parameter group. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore
            max_lr may not actually be reached depending on
            scaling function. Default: 0.006
        step_size (int): Number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch. Default: 2000
        mode (str): One of {triangular, triangular2, exp_range}.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
            Default: 'triangular'
        gamma (float): Constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
            Default: 1.0
        scale_fn (function): Custom scaling policy defined by a single
            argument lambda function, where
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored
            Default: None
        scale_mode (str): {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on
            cycle number or cycle iterations (training
            iterations since start of cycle).
            Default: 'cycle'
        last_batch_iteration (int): The index of the last batch. Default: -1
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> scheduler = torch.optim.CyclicLR(optimizer)
        >>> data_loader = torch.utils.data.DataLoader(...)
        >>> for epoch in range(10):
        >>>     for batch in data_loader:
        >>>         scheduler.batch_step()
        >>>         train_batch(...)
    .. _Cyclical Learning Rates for Training Neural Networks: https://arxiv.org/abs/1506.01186
    .. _bckenstler/CLR: https://github.com/bckenstler/CLR
    """

    def __init__(self, optimizer, base_lr=1e-5, max_lr=6e-5,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [ ]:
def evaluation(dataloader):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs,aux = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [62]:
max_epochs=15
n_freeze=1
n_folds=4
batch_size=64

In [28]:
def train_model(model,train_sampler,valid_sampler):
    early_stopping = EarlyStopping(patience=2, verbose=True)

    optimizer = torch.optim.SGD(model.fc.parameters(), lr=1e-5, momentum=0.99)
    #learn_rate=CyclicLR(optimizer, base_lr=1e-5, max_lr=2e-5,step_size=40, mode='triangular2',gamma=0.99994)
    
    train_ds=eye(train_labels,data_dir+'/train_images/',transform=train_trans)
    valid_ds=eye(train_labels,data_dir+'/train_images/',transform=test_trans)
    
    train=torch.utils.data.DataLoader(train_ds,batch_size=64,num_workers=4,sampler=train_sampler)
    valid=torch.utils.data.DataLoader(valid_ds,batch_size=64,num_workers=4,sampler=valid_sampler)
    #dataiter=iter(train)
    #img,label=next(dataiter)
    #print("\n Shape of image",img.shape)
    #print("\n Shape of label",label[1].item())
    criterion=nn.CrossEntropyLoss()
    
    for epoch in range(max_epochs):
        
       
        for param in model.parameters():
            param.requires_grad = True

                
        print(time.ctime(),'Epoch:',epoch+1)
        loss_arr=[]
        loss_epoch_arr=[]
        model.train()
        for i,(inputs,labels) in enumerate(train,0):
            #inputs=batch['image']
            #labels=batch['label'].view(-1,1)
        #inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            #https://stackoverflow.com/questions/53476305/attributeerror-tuple-object-has-no-attribute-log-softmax
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
            loss_arr.append(loss.item())
        print("\ntraining completed for epoch:",epoch+1,"Validation started at:",time.ctime())
        model.eval()
        val_loss=[]
        no_val_steps=0
        for i,(input,label) in enumerate(valid,0):
            #inputs=batch['image']
            #labels=batch['label'].view(-1,1)
            inputs,labels=inputs.to(device),labels.to(device)
            with torch.no_grad():
                
                output=model(inputs)
            
            loss=criterion(output,labels)
            val_loss.append(loss.item())
            no_val_steps+=1
            #learn_rate.batch_step(np.mean(val_loss))
            #val_loss=val_loss/no_val_steps
        print(f'Epoch {epoch+1} completed {time.ctime()} , train loss: {np.mean(loss_arr):.4f}, valid loss: {np.mean(val_loss):.4f}.')
        early_stopping(np.mean(val_loss),model)
        
        if early_stopping.early_stop:
            print("Early Stopping")
            break
        
    model.load_state_dict(torch.load('checkpoint.pt'))
    return(model)
    
    
    
        
        
    
    
    
    
    
    
    

In [10]:
num_train=len(train_labels)
indices=list(range(num_train))
kf=KFold(n_splits=n_folds,random_state=1000,shuffle=True)

In [29]:
train_idx = []
valid_idx = []

for t, v in kf.split(indices):
    train_idx.append(t)
    valid_idx.append(v)

In [40]:
test_dataset = RetinopathyDatasetTest(
    csv_file='../input/aptos2019-blindness-detection/sample_submission.csv', transform=test_trans)
data_loader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
  


fold_predictions = np.zeros((len(test_dataset), n_folds))
fold_predictions.shape

(1928, 4)

In [63]:
for fold in np.arange(n_folds):
    
    print('Fold:',fold)
    
    # Model    

    model=models.resnet50()
    model.load_state_dict(torch.load("../input/pretrained-pytorch-models/resnet50-19c8e357.pth"))
    final_in_features=model.fc.in_features
    model.fc=nn.Linear(final_in_features,5,bias=True)
    model=model.to(device)
    for param in model.parameters():
        param.requires_grad = False
    
    # Train with early stopping
    
    train_sampler = SubsetRandomSampler(train_idx[fold])
    valid_sampler = SubsetRandomSampler(valid_idx[fold])

    model = train_model(model,train_sampler,valid_sampler)
    
    # Inference for each CV split

    model.eval()
    
    for param in model.parameters():
        param.requires_grad = False
   
    test_preds = np.zeros((len(test_dataset), 1))  
    
   
    
    for i, x_batch in enumerate(data_loader_test):
        x_batch = x_batch["image"]
        pred = model(x_batch.to(device))
        test_preds[i * batch_size:(i + 1) * batch_size] = np.mean(pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1))
                       
    fold_predictions[:,fold] = test_preds.reshape(fold_predictions.shape[0])
    
    del(model, test_preds)
    gc.collect()
    torch.cuda.empty_cache() 
    
fold_predictions_avg = np.argmax(fold_predictions, axis=1)

Fold: 0
Sun Aug 11 06:48:48 2019 Epoch: 1

training completed for epoch: 1 Validation started at: Sun Aug 11 06:54:09 2019
Epoch 1 completed Sun Aug 11 06:55:23 2019 , train loss: 1.0103, valid loss: 0.1234.
Validation loss decreased (inf --> 0.123447).  Saving model ...
Fold: 1
Sun Aug 11 06:56:20 2019 Epoch: 1

training completed for epoch: 1 Validation started at: Sun Aug 11 07:01:42 2019
Epoch 1 completed Sun Aug 11 07:02:52 2019 , train loss: 1.0299, valid loss: 0.1374.
Validation loss decreased (inf --> 0.137440).  Saving model ...


In [ ]:
#  for param in model.parameters():
#         param.requires_grad = False
# test_ds=eye(sample,data_dir+'/test_images/',transform=test_trans)
# test=torch.utils.data.DataLoader(test_ds,num_workers=4,batch_size=batch_size,shuffle=False)
# test_preds = np.zeros((len(test_ds), 1)) 
    
# for i,(data,_) in enumerate(test):
#         print(i)
#         data=data.to(device)
#         pred = model(data)
#         test_preds[i * batch_size:(i + 1) * batch_size] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)
                       
# fold_predictions[:,fold] = test_preds.reshape(fold_predictions.shape[0])

In [ ]:
# %%time
# loss_arr = []
# loss_epoch_arr = []
# max_epochs = 3

# for epoch in range(max_epochs):
#     model.train()
#     for i, (inputs,labels) in enumerate(train_dl,0):
        
#         #inputs, labels = data
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         with torch.set_grad_enabled(True):
#         #https://stackoverflow.com/questions/53476305/attributeerror-tuple-object-has-no-attribute-log-softmax
#             outputs,aux = model(inputs)
#             loss = criterion(outputs, labels)
            
#             loss.backward()
#             optimizer.step()
        
#         loss_arr.append(loss.item())
        
#     loss_epoch_arr.append(loss.item())
        
#     print('Epoch: %d/%d, Train acc: %0.2f, Valid acc: %0.2f' % (epoch, max_epochs, evaluation(train_dl), evaluation(valid_dl)))
    
    
# plt.plot(loss_epoch_arr)
# plt.show()

In [ ]:
# test_ds=eye(sample,data_dir+'/test_images/',transform=test_trans)
# test=torch.utils.data.DataLoader(test_ds,shuffle=True)

In [ ]:
# ## Taken from https://www.kaggle.com/ateplyuk/aptos-pytorch-starter-rnet50

# # Prediction
# predict = []
# model.eval()
# for i, (data, _) in enumerate(test):
#     data = data.cuda()
#     output = model(data)  
#     output = output.cpu().detach().numpy()    
#     predict.append(output[0])

In [ ]:
# np.argmax(predict,axis=1)

In [64]:
sample['diagnosis'] = fold_predictions_avg
sample.head()

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,2
2,006efc72b638,2
3,00836aaacf06,2
4,009245722fa4,2


In [65]:
sample['diagnosis'].value_counts()

2    1344
3     584
Name: diagnosis, dtype: int64

In [ ]:
sample.to_csv("submission.csv",index=False)